<a href="https://colab.research.google.com/github/Kibika/speech-to-text/blob/speech_to_text/notebooks/speech_2_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install dvc

     |████████████████████████████████| 638 kB 7.3 MB/s 
     |████████████████████████████████| 208 kB 73.3 MB/s 
     |████████████████████████████████| 529 kB 52.6 MB/s 
     |████████████████████████████████| 49 kB 7.3 MB/s 
     |████████████████████████████████| 76 kB 5.9 MB/s 
     |████████████████████████████████| 170 kB 65.9 MB/s 
     |████████████████████████████████| 118 kB 69.3 MB/s 
     |████████████████████████████████| 40 kB 7.0 MB/s 
     |████████████████████████████████| 4.6 MB 44.2 MB/s 
     |████████████████████████████████| 296 kB 70.8 MB/s 
     |████████████████████████████████| 108 kB 70.8 MB/s 
     |████████████████████████████████| 44 kB 3.2 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 201 kB 68.0 MB/s 
     |████████████████████████████████| 2.6 MB 51.6 MB/s 
     |████████████████████████████████| 64 kB 3.8 MB/s 
     |████████████████████████████████| 51 kB 7.4 MB/s 
     |███████████████████████

In [3]:
pip install torchaudio

     |████████████████████████████████| 1.9 MB 4.2 MB/s 


In [4]:
! pip install SpeechRecognition

     |████████████████████████████████| 32.8 MB 31 kB/s 


In [ ]:
!pip install mlflow --quiet
import mlflow
import os
from getpass import getpass

os.environ['MLFLOW_TRACKING_USERNAME'] = input('Enter your DAGsHub username: ')
os.environ['MLFLOW_TRACKING_PASSWORD'] = getpass('Enter your DAGsHub access token: ')
os.environ['MLFLOW_TRACKING_PROJECTNAME'] = input('Enter your DAGsHub project name: ')

mlflow.set_tracking_uri(f'https://dagshub.com/' + os.environ['MLFLOW_TRACKING_USERNAME'] 
                        + '/' + os.environ['MLFLOW_TRACKING_PROJECTNAME'] + '.mlflow')

with mlflow.start_run(run_name="MLflow on Colab"):
  mlflow.log_metric("m1", 2.0)
  mlflow.log_param("p1", "mlflow-colab")

     |████████████████████████████████| 14.4 MB 64 kB/s 
     |████████████████████████████████| 636 kB 54.2 MB/s 
     |████████████████████████████████| 54 kB 4.0 MB/s 
     |████████████████████████████████| 1.1 MB 45.9 MB/s 
     |████████████████████████████████| 146 kB 68.7 MB/s 
     |████████████████████████████████| 79 kB 9.8 MB/s 
     |████████████████████████████████| 75 kB 5.8 MB/s 
     |████████████████████████████████| 68 kB 9.0 MB/s 


KeyboardInterrupt: ignored

In [ ]:
# See your experiments table inside Colab!
import IPython
display(IPython.display.IFrame("https://dagshub.com/"+ os.environ['MLFLOW_TRACKING_USERNAME'] 
                        + '/' + os.environ['MLFLOW_TRACKING_PROJECTNAME'] + "/experiments/#/",'100%',600))

KeyError: ignored

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# cd drive/MyDrive/speech

In [1]:
! git clone https://github.com/getalp/ALFFA_PUBLIC.git

Cloning into 'ALFFA_PUBLIC'...
remote: Enumerating objects: 66024, done.
remote: Total 66024 (delta 0), reused 0 (delta 0), pack-reused 66024
Receiving objects: 100% (66024/66024), 6.54 GiB | 24.88 MiB/s, done.
Resolving deltas: 100% (12337/12337), done.
Checking out files: 100% (56925/56925), done.


In [ ]:
# ! git clone https://github.com/Kibika/speech-to-text.git

In [ ]:
# ! git init 

In [ ]:
# #connect to github
# # configuring git for user account
# ! git config --global user.name "Kibika" #<github user id>
# ! git config --global user.email "steshykibika@gmail.com" #<github email id>


# Read the dataset from github
Thhe dataset is contained in several folders in a git repo. The repo containing the data is cloned and os.walk is used to extract the .wav files from the subdirectories.

In [5]:
import wave
import os
import speech_recognition as sr
import librosa
import csv
import pandas as pd
import numpy as np

In [6]:
# dataset = pd.read_csv("/content/drive/My Drive/Bank Statements/newcustomerdata.csv")
Path_to_train = "/content/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav"
Path_to_test = "/content/ALFFA_PUBLIC/ASR/SWAHILI/data/test/wav5"
Path_to_train_labels = "/content/ALFFA_PUBLIC/ASR/SWAHILI/data/train/text"
Path_to_test_labels="/content/ALFFA_PUBLIC/ASR/SWAHILI/data/test/text"
OUTPUTFILE = '/content/speech-to-text/data/speech_features.csv'
# speech_data_nofeatures=[]


In [7]:
#extract all .wav from subdirectories
def get_file_paths(dirname):
    file_paths = []  
    for root, directories, files in os.walk(dirname):
        for filename in files:
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)  
    return file_paths  

In [8]:
#create a folder with all the train files
import shutil
in_path = Path_to_train
out_path = "/content/ALFFA_PUBLIC/ASR/SWAHILI/data/train/all_files"
 
for dirpath, dirnames, filenames in os.walk(in_path):
    for filename in filenames:
        if filename.endswith(".wav"):
             src = os.path.join(dirpath, filename)
             dest = os.path.join(out_path, filename)
             shutil.copy2(src, dest)

In [ ]:
#Process the data
# def process_file(file):
#     r = sr.Recognizer()
#     a = ''
#     with sr.AudioFile(file) as source:
#         audio = r.record(source)    
#         try:
#             a =  r.recognize_google(audio)        
#         except sr.UnknownValueError:
#             a = "Google Speech Recognition could not understand audio"
#         except sr.RequestError as e:
#             a = "Could not request results from Google Speech Recognition service; {0}".format(e)  
#     return a

In [ ]:
# def main():
#     files = get_file_paths(Path_to_train)                 # get all file-paths of all files in dirname and subdirectories
#     for file in files:                              # execute for each file
#         (filepath, ext) = os.path.splitext(file)    # get the file extension
#         file_name = os.path.basename(file)          # get the basename for writing to output file
#         if ext == '.wav':                           # only interested if extension is '.wav'
#             a = process_file(file)                  # result is returned to a
#             with open(OUTPUTFILE, 'a') as f:        # write results to file
#                 writer = csv.writer(f)
#                 writer.writerow(['file_name','google'])
#                 writer.writerow([file_name, a])        

In [ ]:
# #put extracted .wav files into a dataframe
# speech_df = pd.DataFrame (get_file_paths(Path_to_train),columns=['recording'])
# speech_df.head()

In [ ]:
# #extract content of text file(labels)
# text_labels=pd.DataFrame(pd.read_csv(Path_to_train_labels, sep=',',header=None, names=['file_path'])['file_path'].str.split('\t',1).tolist(),
#                                  columns = ['file_path','text'])
# text_labels.head()

In [9]:
pd.set_option('display.max_colwidth',None)

Read in the labels data from .txt file

In [10]:
#merge dataframes based on the name of the file path use metadata

#read text from every transcription audio
def read_text( text_path):
    text = []
    with open(text_path) as fp:
        line = fp.readline()
        while line:
            text.append(line)
            line = fp.readline()
    return text



obtain the data folder name and texts from the .txt file

In [11]:
label=[]
transcriptions = []
for t in read_text(Path_to_train_labels):
    name=t.split("\t")[1]
    # name=name.replace('(', '')
    # name=name.replace(')', '')
    name=name.replace('\n','')
    # name=name.replace(' ','')
    text=t.split("\t")[0]
    # text=text.replace("\t","")
    # name_to_text[name]=text
    label.append(text)
    transcriptions.append(name)


In [12]:
#get audio path , every path must corespond to transcription , get the transprion in the doc and append to audio path 
audio_path=[0]*len(transcriptions)
for d in get_file_paths(Path_to_train):
    _d = d.strip(".wav")
    sp = _d.split("/")[9]
    index = label.index(sp)
    audio_path[index] = d

In [13]:
#calculate duration 
duration_of_recordings=[]
for d in audio_path:
    audio, fs = librosa.load(d, sr=None)
    duration_of_recordings.append(float(len(audio)/fs))

In [14]:
speech_data=pd.DataFrame({'key': audio_path,'text': transcriptions, 'duration':duration_of_recordings})
speech_data.head()

,key,text,duration
0,/content/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-05-20101106/SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part10.wav,rais wa tanzania jakaya mrisho kikwete,3.14
1,/content/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-05-20101106/SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part100.wav,yanayo andaliwa nami pendo pondo idhaa ya kiswahili,3.10
2,/content/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-05-20101106/SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part101.wav,inayokutangazia moja kwa moja kutoka jijini dar es salaam tanzania,3.65
3,/content/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-05-20101106/SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part102.wav,juma hili bara la afrika limeshuhudia raia wa nchi za niger,3.90
4,/content/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-05-20101106/SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part103.wav,wakipiga kura ya maoni ilikufanya mabadiliko ya,2.94


# Visualization

# Data Preprocessing

In [ ]:
recognizer = sr.Recognizer()

In [ ]:
# set energy threshold
# audios that are below this threshold will be considered silent
recognizer.energy_threshold = 300

# Resize and standardize
We resize all the audio samples to have the same length by either extending its duration by padding it with silence, or by truncating it.

Standardize the sample rate.


In [15]:
# used to load audio file
#specifying sample rate will resize all the files i.e Audio will be automatically resampled to the given rate
class Loader:
  def __init__(self, sample_rate,duration,mono):
    self.sample_rate=sample_rate
    self.duration=duration
    self.mono=mono

  def load_signal(self, filepath):
    signal=librosa.load(filepath,
                        sr=self.sample_rate,
                        duration=self.duration,
                        mono=self.mono)[0]    #librosa returns 2D array (signal,sample_rate) pick the signal here
    return signal

  def load_sample_rate(self,filepath):
    sample_rate=librosa.load(filepath,
                        sr=self.sample_rate,
                        duration=self.duration,
                        mono=self.mono)[1]    #librosa returns 2D array (signal,sample_rate) pick the sample rate here
    return sample_rate

  def load(self,filepath):     #get both signal and sample rate in one
    aud=librosa.load(filepath,
                        sr=self.sample_rate,
                        duration=self.duration,
                        mono=self.mono)
    return aud


In [ ]:
#resizes audios to have same length
class Padder:
  def __init__(self,mode="constant"):
    self.mode=mode
  def left_pad(self,array,num_missing_items):
    padded_array=np.pad(array,
                        (num_missing_items, 0),
                        mode=self.mode)
    return padded_array
  def right_pad(self,array,num_missing_items):
    padded_array=np.pad(array,
                        (0,num_missing_items),
                        mode=self.mode)
    return padded_array
  



In [ ]:
# Resample for stereo
class Resampling:
  def __init__(self):
    self.loader=None
    self.newsr=44100

  def resample(self):
    sig, sr = self.loader.load(filepath)

    if (sr == self.newsr):
      # Nothing to do
      return self.loader.load(filepath)

    num_channels = sig.shape[0]
    # Resample first channel
    resig = torchaudio.transforms.Resample(sr, self.newsr)(sig[:1,:])
    if (num_channels > 1):
      # Resample the second channel and merge both channels
      retwo = torchaudio.transforms.Resample(sr, self.newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))

In [ ]:
#mono to stereo conversion
class Convert:
  def __init__(self):
    self.loader=None
    self.channel = 2
  def rechannel(self, filepath):
    sig, sr = self.loader.load(filepath)

    if (sig.shape[0] == self.channel):
      # Nothing to do
      return self.loader.load(filepath)

    if (self.channel == 1):
      # Convert from stereo to mono by selecting only the first channel
      resig = sig[:1, :]
    else:
      # Convert from mono to stereo by duplicating the first channel
      resig = torch.cat([sig, sig])

    return ((resig, sr))

In [ ]:
class Saver():
  pass

In [ ]:
class PreprocessingPipeline:
  '''Processes audio files in a directory by applying the following steps
    1. Load the data, convert to mono and resample sampling rate
    2. Pad the audio
  '''
  def __init__(self):
    self.padder=None
    # self.saver=None
    self._loader=None
    # self._num_expected_samples=None
    self.resampling=None
    self.convert=None

 
  # def loader(self):
  #   return self._loader


 
  # def loader(self,loader):
  #   self.loader=loader
  #   self._num_expected_samples=int(loader.sample_rate*loader.duration)



  def process(self,audio_files_directory):
    for root, directories, files in os.walk(audio_files_directory):
        for filename in files:
            filepath = os.path.join(root, filename)
            self._process_file(filepath)
            print(f"Processed file {filepath}")
    
  def _process_file(self,filepath):
    signal=self.loader.load_signal(filepath)
    if self._is_padding_necessary(signal):
      signal=self._apply_padding(signal)

  def _is_padding_necessary(self,signal):
    num_expected_samples=int(loader.sample_rate*loader.duration)
    if len(signal) < num_expected_samples:
      return True
    return False

  def apply_padding(self,signal):
    num_missing_samples=self._num_expected_samples - len(signal)
    padded_signal = self.padder.right_pad(signal, num_missing_samples)
    return padded_signal


In [ ]:
DURATION=0.74
SAMPLE_RATE=22050
MONO=False

In [ ]:
loader=Loader(SAMPLE_RATE, DURATION, MONO)
padder=Padder()

In [ ]:
# import sys
# sys.setrecursionlimit(5000)
preprocessing_pipeline=PreprocessingPipeline()
preprocessing_pipeline.loader=loader
preprocessing_pipeline.padder=padder

In [ ]:
preprocessing_pipeline.process(Path_to_train)

Streaming output truncated to the last 5000 lines.
Processed file /content/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-15-20110207/SWH-15-20110207_16k-emission_swahili_15h00_-_16h00_tu_20110207_part357.wav
Processed file /content/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-15-20110207/SWH-15-20110207_16k-emission_swahili_15h00_-_16h00_tu_20110207_part292.wav
Processed file /content/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-15-20110207/SWH-15-20110207_16k-emission_swahili_15h00_-_16h00_tu_20110207_part198.wav
Processed file /content/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-15-20110207/SWH-15-20110207_16k-emission_swahili_15h00_-_16h00_tu_20110207_part168.wav
Processed file /content/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-15-20110207/SWH-15-20110207_16k-emission_swahili_15h00_-_16h00_tu_20110207_part99.wav
Processed file /content/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-15-20110207/SWH-15-20110207_16k-emission_swahili_15h00_-_16h00_tu_20110207_part196.wav
Processed file /cont